# Tutorial 3: How to select the best settings and advanced options
## flowTorch workshop 29.09.2025 - 02.10.2025

### Outline
here: 
- some features and advanced options for $S^3$
- influence of different settings and how to select them: uniform levels, delta level constraint, pre_select, n_cells_iter_*

In [ ]:
import sys
import numpy as np
import torch as pt

from stl import mesh
from os.path import join
from os import environ, system

environ["sparseSpatialSampling"] = ".."
sys.path.insert(0, environ["sparseSpatialSampling"])

from sparseSpatialSampling.export import ExportData
from sparseSpatialSampling.sparse_spatial_sampling import SparseSpatialSampling
from sparseSpatialSampling.geometry import CubeGeometry, SphereGeometry

In [ ]:
# same as tutorial 1
# define load paths to the CFD data
load_path = join("..", "..", "flow_data", "run", "cylinder_2D_Re100")

# here we use the approximated metric field as stopping criterion
# how much of the metric within the original grid should be captured at least
min_metric = 0.75

# define the path to where we want to save the results and the name of the file
save_path = join("..", "run", "tutorials", "tutorial_3")
save_name = "cylinder2D_metric_{:.2f}".format(min_metric)
# now we can load the data. Since we used OpenFOAM, we can use the load_foam_data function provided by S^3.
# Otherwise, we have to use flowtorch dataloaders directly, refer to the flowtorch documentation

# define boundaries of the masked domain for the cylinder, here we want to load the full domain
bounds = [[0, 0], [2.2, 0.41]]  # [[xmin, ymin], [xmax, ymax]]

# load the CFD data, we want to compute the metric based on the velocity in the quasi-steady state, so omit the first 4 seconds
field, coord, _, write_times = load_foam_data(load_path, bounds, field_name="U", t_start=4, scalar=False)

In [ ]:
# now we compute a metric. in this case, we just use the temporal mean of the abs. velocity vector
metric = pt.mean(field.abs().sum(1), 1)

# create geometry objects for the domain and the cylinder
# we don't want to refine the domain boundaries, so keep all the optional arguments as default
domain = CubeGeometry("domain", True, bounds[0], bounds[1])
geometry = SphereGeometry("cylinder", False, [0.2, 0.2], 0.05, refine=True, min_refinement_level=9)

# TODO: delta_level constraint
s_cube = SparseSpatialSampling(coord, metric, [domain, geometry], save_path, save_name, "cylinder2D", min_metric=min_metric, n_jobs=4,
                              max_delta_level=True)